**IOC 11 – Credential Harvesting via Embedded PDF Redirect**
**1. Attacker Description and Analogy**
Technical Description:

The attacker embedded a malicious link into a PDF file designed to look like a routine document. When the user opened the file with Adobe Reader, an invisible hyperlink—overlaid onto ordinary-looking text—was triggered upon user click. This link redirected through a URL shortener and then connected to a typosquatted domain designed to look like a Microsoft login page. The PDF's behavior caused Adobe Reader to spawn a child browser process (Microsoft Edge), which then initiated a DNS lookup and HTTP POST to the attacker-controlled server.

No malware was written to disk. This was a fileless credential harvesting attempt, using user interaction and redirect obfuscation to evade traditional detections.

Analogy:

This attacker didn’t break into a house—they sent an official-looking envelope. Inside was a letter inviting the recipient to "redeem your secure document." The recipient opened it and followed the included map (the hyperlink). That map led not to their secure vault, but to a fake bank window built by the attacker. It looked official, complete with glass, signage, and a uniformed employee. But it wasn’t real. The user gave away their credentials, thinking it was a routine login.

Cast of Characters
Adobe Reader (AcroRd32.exe) – Trusted Mail Carrier: Delivers routine documents but can be manipulated.

PDF File – Deceptive Envelope: Seemingly benign, but conceals hidden instructions.

Microsoft Edge (msedge.exe) – Trusted Doorway: The browser opens at the request of Adobe Reader.

trackupdate[.]info –  – Redirect Signpost - a misleading signpost that reroutes users to the wrong destination to disguise the true target. : Redirector that masks the attacker’s real destination.

micros0ft-verify[.]com – Spoofed Bank Window: A typosquatted phishing site mimicking a Microsoft login.

EDR Platform – Motion Sensor and Investigator: Detected and correlated behavior across tools.

**2. Source of IOC (Telemetry Origin)**
Technical:

The IOC was triggered by the organization's Endpoint Detection and Response (EDR) platform. The system observed Adobe Reader (AcroRd32.exe) opening a PDF file, which immediately spawned Microsoft Edge (msedge.exe). This was flagged because document readers do not typically invoke browsers. DNS logs showed resolution of trackupdate[.]info, followed by a redirect to micros0ft-verify[.]com. A suspicious HTTP POST followed. PDF metadata confirmed the presence of a hidden link embedded into a clickable text layer. EDR correlation logic flagged this as an anomalous process chain likely representing credential harvesting via redirection.

Softened Interpretation:

The security team spotted a strange pattern: a mailman (Adobe Reader) delivered a letter, and the moment the recipient opened it, they were escorted by a trusted guide (Edge) to a strange building. The team checked the visitor logs (EDR telemetry) and saw that the guide went to two locations (redirect + phishing site). Further analysis of the envelope (PDF metadata) revealed a hidden instruction. This wasn’t an ordinary delivery—it was a baited tour.

**3. Triage Framework Declaration**
Triage Type: Social Engineering / Obfuscated Link

Canonical Toolset Used (with contextual descriptions):

Artifact Capture: The malicious PDF was isolated and its metadata inspected using forensic tools.

URL Inspection: The embedded hyperlink was extracted and expanded to reveal a two-stage redirection chain.

User Investigation: Interview confirmed that the user believed the document was from an HR system and clicked without suspicion.

Isolation & Blocking: The domain was added to the block list, and the affected host was temporarily segmented.

Softened Interpretation:

We opened the original envelope (Artifact Capture) and saw a strange ink signature inside. We unfolded the hidden map (URL Inspection), and it showed a winding route. We talked to the person who opened it (User Investigation), and they said it looked official. We then blocked the route on all maps going forward (Isolation & Blocking).

**4. OS Layer with Key Behaviors**
Mapped Host OS Layer: Layer 6 – Network Communication Context

Adobe Reader launched Edge in response to a user action. This triggered an outbound DNS request and HTTP POST. These behaviors map to Layer 6 (network communication context) because they represent direct communication between a host process and external resources via the network stack. The absence of file drops or persistent code confirms this was a fileless delivery and exfiltration vector.

Softened Interpretation:

The house didn’t show signs of a break-in—no broken locks, no tampered doors. But someone inside made a phone call to a scammer pretending to be the bank (HTTP POST via browser). They used a trusted landline (Edge) connected through a routine system (Layer 6). No footprints left, but a conversation was recorded.

**5. Cross-Layer Interaction Pivots**
While this case was not formally triaged via the Host Triage Protocol, behavior observed can be logically backtracked through host layers:

Layer 1 → Layer 6: Adobe Reader executed (Layer 1), resulting in outbound traffic (Layer 6).

Layer 6 → Layer 5: DNS and HTTP behavior raised detection via EDR’s behavioral telemetry (Layer 5).

No persistence or privilege escalation occurred; thus, Layer 2–4 were not invoked.

Softened Interpretation:

The attacker rang the phone (Layer 6), but they never moved into the house (no Layer 2–4 action). The system’s security camera (EDR – Layer 5) heard the voice and flagged it. The event began with a legitimate visitor (Layer 1), but ended in a deceptive call.

**6. OSI Layer Relevance**
Layer 7 (Application): The PDF file triggered actions at the application level.

Layer 4 (Transport): The connection was sent via TCP-based HTTP POST.

Layer 3 (Network): IP communication confirmed.

Layer 2/1: Not involved in this case.

Softened Interpretation:

This was all about the words spoken (Layer 7), how the words traveled (TCP – Layer 4), and where they were routed (IP – Layer 3). No one cut the phone lines or tampered with wires (Layer 2/1 untouched).

**7. Attacker Behavior Interpretation**
This attacker used minimal signals and maximum user trust. They disguised a malicious link within an otherwise normal document, relying on visual trust cues and redirection to bypass detection. The intent was to gather credentials without dropping malware or invoking known privilege escalation techniques. The strategy was clean, social, and silent.

Softened Interpretation:

This wasn’t a battering ram. It was a phony invitation tucked inside a legit envelope, handed off by a well-dressed actor. The system didn’t realize it was fooled until it tracked the caller ID history.

**8. Defender Action Summary**  
PDF sample extracted and sandboxed.

Hyperlink decoded and threat domain identified.

DNS logs confirmed outbound contact.

Host segmented and user notified.

EDR rule updated to flag future Adobe Reader → browser chains.

Softened Interpretation:

We opened the suspicious envelope (PDF sample extracted and sandboxed), unfolded its map (hyperlink decoded), and saw that it led to a dangerous part of town (identified threat domain). Then we checked the phone records and confirmed the resident had already made the call (DNS logs showed outbound contact). We posted a “Do Not Enter” sign on their door (host was segmented and blocked from further contact), and made sure the resident knew never to trust a letter like that again (user was notified of phishing tactics). Finally, we updated our neighborhood watch protocols (EDR rule updated to flag future Adobe Reader → browser process chains) so that if anyone else receives a similar envelope, the alarms will go off immediately.


**9. Attacker Strategy Notes**
Used visual trust and document familiarity to lower suspicion.

Leveraged user interaction as the payload trigger.

Chose a fileless method to avoid AV detection.

Created a typosquatted phishing domain that closely resembled a legitimate Microsoft service.

Escaped detection by exploiting lack of DNS content filtering and redirect logic.Lack of DNS content filtering means the organization’s DNS infrastructure did not check domain destinations against a threat intelligence list or enforce category-based blocking. As a result, the user’s system was allowed to resolve and connect to a newly registered, suspicious domain without restriction.

Redirect logic refers to how the attacker’s phishing domain immediately forwarded the user to a second malicious site after the initial connection. This two-stage redirection chain allowed the attacker to hide the final payload destination and bypass simple domain-based filters that only inspect the first hop.



Softened Interpretation:The attacker didn’t need to break windows or pick locks—they slipped an envelope under the door that looked exactly like something you'd expect (used visual trust and document familiarity). Inside, the envelope contained a button disguised as a polite invitation, which only worked if you pressed it yourself (relied on user interaction as the trigger). There was no traceable fingerprint on the envelope—just static electricity and sleight of hand (used fileless methods to avoid AV detection). The return address on the envelope was nearly identical to Microsoft’s, but just off by a letter, enough to fool the untrained eye (typosquatted phishing domain). And the postal service? It didn’t screen outgoing or redirected mail, letting the trickster bounce the letter through shadowy side alleys (exploited lack of DNS filtering and redirect logic) with no one the wiser.



**10. Final Analyst Reflection and Softened Narrative**
This case reminds us that fileless doesn’t mean harmless. The system didn’t detect malware because there was none. But the chain of trust was subverted—a passive application invoked a trusted browser to access a fake site.

To stop this, we must treat unexpected application-to-browser handoffs as potentially malicious, especially when they result in DNS resolutions to obscure domains.

Softened Interpretation:

A trusted friend handed you an envelope. You opened it, made a call, and gave up your password. But the envelope wasn’t from your friend—it only looked like it was. You weren’t hacked. You were tricked into unlocking the door yourself.

**11. Who’s Who – Object Role Clarification**
Technical Component	Narrative Role	Definition
Adobe Reader	Trusted Mail Carrier	Opened the PDF document that launched a browser
PDF File	Deceptive Envelope	Contained embedded link triggering the redirect
Edge Browser	Trusted Guide	Browser launched from PDF to visit link
Redirect URL	Map Booth	Obfuscated the destination by redirecting
Phishing Domain	Fake Bank Window	Typosquatted Microsoft login page
EDR	Motion Sensor	Detected and correlated behavior across tools

**CONCLUSION**

IOC 11 – Malicious PDF Redirect via Embedded Link

Overview
This case study examines an obfuscated phishing attempt embedded in a PDF file. The attack relied on user interaction to initiate a hidden redirect to a typosquatted phishing domain. No malware was dropped, no process was overtly malicious, and the PDF appeared benign—until it triggered a covert network egress through a legitimate application (Adobe Reader). The attacker’s goal was credential harvesting via social engineering and DNS evasion.

Key IOC Highlights
Source of IOC: EDR (Endpoint Detection and Response) alert on suspicious PDF→browser behavior chain

Triage Type: Social Engineering / Obfuscated Link

Process Chain: AcroRd32.exe (Adobe Reader) → msedge.exe (Microsoft Edge)

Indicator Artifacts: DNS query to typosquatted domain, hyperlink embedded in PDF metadata

Cross-Layer Activity: Involved OS Layer 1 (process), Layer 6 (network egress), and Layer 5 (EDR log activity)

Attacker Tactics
Fileless delivery via trusted application

User-triggered execution

Exploitation of weak DNS filtering

Redirect chaining to mask destination

Deceptive social engineering embedded in legitimate document

Defender Response
PDF isolated and sandboxed for analysis

Embedded URL decoded and linked to malicious infrastructure

DNS logs used to confirm outbound attempt

EDR detection rule updated to flag similar behavior chains

User notified and workstation segmented from broader network

Educational Value
This IOC emphasizes the subtlety of real-world phishing techniques that do not rely on payloads, but on user behavior and overlooked telemetry paths (e.g., DNS, PDF metadata, and browser spawning). It demonstrates the necessity of:

DNS content filtering

Behavioral correlation rules

Endpoint telemetry interpretation

Deep cross-layer investigative practices
